In [ ]:
## 安装调用大模型的openai库和读pdf的pypdf库

# !pip install openai pypdf

In [ ]:

import openai
import json

# API密钥
API_TOKEN = ""  # 此处需要填写对应的api密钥
MODEL_NAME = 'deepseek-v3'  # 选择要调用的模型名称

# 请求URL
BASE_URL = "https://ng.115.zone/v1/"

# 创建一个LLM顾问实例对象
llm_client = openai.OpenAI(api_key=API_TOKEN, base_url=BASE_URL)


# 不要修改这个函数
# 此函数的作用是发送LLM调用请求
def llm_generate(messages: list[dict[str, str]], model_name=MODEL_NAME):
    """
    发送OpenAI格式的大模型请求

    参数:
    - messages: 包含对话历史的字典列表，每个字典包含role和content键,role为user或assistant
    - model_name: 要调用的模型名称

    返回:
    - 模型生成的回复内容
    """
    try:
        # 发送请求
        response = llm_client.chat.completions.create(
            model=model_name,
            messages=messages,
            stream=False
        )

        # 返回模型生成的内容
        return response.choices[0].message.content

    except Exception as e:
        print(f"请求发生错误: {e}")
        return None

In [ ]:
# 简单尝试提问的问题
question = "西安电子科技大学新手入学实验班选拔考试考哪些科目，所占的比重如何"

print("## 问题：", question)
## 模型推理代码⬇️

# 表明身份是用户user,内容是对应的问题
messages = [
    {"role": "user", "content": question},
]

# 调用llm_generate来生成内容并输出
content = llm_generate(messages)
print(content)

---

# 开始合成数据

## 准备训练文件
在data目录中我们寻找了若干文件对其文本进行读取操作，并且进行了简单的清洗

### 将PDF政策文件提取为TXT

In [ ]:
## 读取文件
from pathlib import Path
from docx import Document
from pypdf import PdfReader

# 项目 data 文件夹路径
data_folder = Path("data2")
output_file = "output2.txt"

def read_docx(file_path):
    """读取 Word 文档内容"""
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def read_pdf(file_path):
    """读取 PDF 文档内容"""
    text = []
    reader = PdfReader(file_path)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:  # 避免空页导致 None
            text.append(page_text)
    return "\n".join(text)

def main():
    all_texts = []

    # 遍历 data 文件夹下所有文件（非递归）
    for file in sorted(data_folder.iterdir()):
        if file.suffix.lower() == ".docx":
            print(f"读取 Word 文件: {file.name}")
            all_texts.append(read_docx(file))
        elif file.suffix.lower() == ".pdf":
            print(f"读取 PDF 文件: {file.name}")
            all_texts.append(read_pdf(file))
        else:
            print(f"跳过文件: {file.name}（非 docx/pdf）")

    # 保存到 output.txt
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n\n".join(all_texts))

    print(f"所有文档内容已保存到 {output_file}")

if __name__ == "__main__":
    main()

### 对txt文件数据进行简单处理

In [ ]:
import re

input_file = "output2.txt"            # 输入文本文件
output_file = "output_cleaned2.txt"   # 输出文本文件

# 匹配页尾标注：中文破折号 — 数字 — 或 英文破折号 - 数字 -
page_mark_pattern = re.compile(r"^(?:—|-)\s*\d+\s*(?:—|-)$")

def clean_text(file_path):
    cleaned_text = []

    with open(file_path, "r", encoding="utf-8") as f:
        # 去掉空行和首尾空格
        lines = [line.strip() for line in f if line.strip()]

    for line in lines:
        # 删除页尾标注
        if page_mark_pattern.match(line):
            continue
        cleaned_text.append(line)

    # 合并所有行为一行
    return "".join(cleaned_text)

def main():
    cleaned_text = clean_text(input_file)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(cleaned_text)

    print(f"处理完成，结果已保存到 {output_file}")

if __name__ == "__main__":
    main()


In [ ]:
with open("output_cleaned2.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(len(text), "characters")

## 构建一个随机抽取函数

从导出的TXT中随机抽取一段文字（默认为抽取200字）

In [ ]:
import random

# 一个阅读pdf文字并且转pdf的函数
# 随机抽取文段，以此来让大模型根据这个文段进行提问

def random_text_chunk(txt_path, chunk_length=800):
    """
    从指定txt文件中随机读取一个长度为 chunk_length 个汉字的文本片段。
    如果文本长度不足，则直接返回全文。
    """
    with open(txt_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 如果文本不够长，直接返回全部内容
    if len(text) <= chunk_length:
        return text

    # 否则随机选取一个起始位置
    start = random.randint(0, len(text) - chunk_length)
    chunk = text[start:start + chunk_length]

    return chunk


text = random_text_chunk("output_cleaned2.txt")
print(text)

## 让模型根据文本提问

In [ ]:
## 用于提问题的提示词⬇️

# 问答前提，可以自定义prompt文本

prompt = "根据下面的内容提取新生可能进行提问的重要信息，然后以新生的口吻提一个问题，注意该问题的答案必须在我提供的片段当中，注意你只返回问题，不要返回任何额外的信息或回答"

# llm是作为system角色回答的
## 模型推理代码⬇️
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "以下是学校规定或者通知要求：\n\n" + text},
]
content = llm_generate(messages)
print(content)

## 构建模型提问函数

为了提取出文字，我们使用input长度来砍掉用于模型生成的文本段

In [ ]:

def question_model_generate(text):
    """
    根据提供的PDF文本内容（如校规），调用模型生成一个以学生口吻提出的问题，
    该问题的答案必须能从提供的文本中找到。

    参数:
    - text (str): 从PDF中提取的文本内容，作为生成问题的依据。

    返回:
    - str: 模型根据参考文本生成的学生提问。
    """
    messages = [
        {"role": "system", "content": "请以西电新生的口吻提出一个自然合理，符合逻辑的问题，问题的答案必须能在我所提供的校规片段中找到。不要包含情景描述描述，例如自称我刚入学，对别人称呼学长等，只输出一个问题，不输出解释或答案。"},
        {"role": "user", "content": text},
    ]
    content = llm_generate(messages)
    return content


# 测试函数是否正常工作
question = question_model_generate(text)
print(question)

## 构建模型回答函数

In [ ]:
# 根据提出的问题生成回答

def answer_model_generate(question, text):
    """
    根据用户提出的问题和提供的PDF文本内容（如校规），调用模型生成回答。

    参数:
    - question (str): 用户提出的问题。
    - text (str): 从PDF中提取的文本内容，作为回答问题的依据。

    返回:
    - str: 模型根据问题和参考文本生成的回答。
    """
    # 这里注意，要将参考文本输入到system prompt当中
    messages = [
        {"role": "system",
         "content": "你是西电新生百事通助手，请根据学生提出的单个问题，以及下面我提供的校规或公告内容，给出准确、完整的回答。回答必须严格依据校规内容，不加入校外信息。只输出回答内容，不输出提问或额外说明，生成的同时请你再次检查是否符合我的要求以及问题是否合理。\n\n" + text},
        {"role": "user", "content": question},
    ]
    content = llm_generate(messages)
    return content


# 测试函数是否正常工作
answer = answer_model_generate(question, text)
print(answer)

## 开始通过循环生成数据

In [ ]:
import time
question_num = 120  # 控制生成的数据数量

GENERATE_QA_DATA = []
for i in range(question_num):
    print(f"############# 生成第{i}个问题")
    # 随机抽取一段文本
    text = random_text_chunk("output_cleaned2.txt")
    # print(f"#### 抽取的文本\n{text}")
    # 模型生成一个问题
    question = question_model_generate(text)
    print("### 问题：")
    print(question)
    # 模型再给一个答案
    answer = answer_model_generate(question, text)
    print("### 回答：")
    print(answer)
    time.sleep(1.5)
    GENERATE_QA_DATA.append((question, answer))

## 以Alpaca数据集格式保存到本地

In [ ]:
import json


file_path = 'dataset.json'  # 输出文件路径

# 用列表收集所有数据
data_list = []

for question, answer in GENERATE_QA_DATA:
    alpaca_data = {
        "instruction": question,
        "input": "",
        "output": str(answer)  # 保证 output 为字符串，避免 object 类型报错
    }
    data_list.append(alpaca_data)

# 写入标准 JSON 数组文件，并美化格式
with open(file_path, 'a', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"数据已保存至 {file_path}")



## 接下来的作业本

尝试修改上一讲的代码，来微调新的数据集